<a href="https://colab.research.google.com/github/iceman67/-Python/blob/master/Anomaly_Detection_Prob_ASOS_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
URL = "https://raw.githubusercontent.com/iceman67/-Python/master/weather-119-10-large.csv"

* CSV 파일을 읽어 데이터프레임으로 구성

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv(URL)
data.head()



## 메타데이터 
* sumSsHr : 합계 일조 시간
* sumRn : 일강수량
* sumLrgEv : 합계 대형증발량
* avgRhm : 평균 상대습도

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.describe()

## 결측치 처리

* 컬럼 별 결측치 비율을 확인한다
* 결측치 처리 

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().sum()/len(data) * 100

* 결측치 처리방법 :  컬럼 중 하나라도 결측치가 있다면 행(row) 제거

In [ ]:
data = data.dropna()

## 상관분석
> 평균기온과 기압과의 관계

* 산포도를 통해 두변수 평균기온과 기압의 상관 파악
* 정량화를 위해 상관 계수를 확인함 (corr)

* 상관분석은 숫자로 표현 가능한 연속형 자료값에서만 가능함
* -1과 1까지의 값으로 상관관계의 크기를 나타냄
* 증가하는 방향성에 대한 인과관계를 의미하지 않으며, 단지 연관성의 정도만을 의미함 



In [ ]:
avg_ta = np.array(data['avgTa'])
avg_pa = np.array(data['avgPa'])

corr = np.corrcoef(avg_ta, avg_pa)
corr

In [ ]:
plt.scatter(avg_ta, avg_pa)
plt.xlabel("Temperature (C)")
plt.ylabel("Pressure (Hpa)")

* corr메서드는 각 열 간의 상관 계수를 반환하는 메서드임

In [ ]:
data.corr()

In [ ]:
sub_data = data[ ["avgTa", "minTa", 'maxTa'] ]


In [ ]:
sub_data.corr()

In [ ]:
plt.figure()
plt.scatter(data['avgTa'], data['minTa'])
plt.title("avgTa vs. minTa")
plt.show()

In [ ]:
sub_data = data[ ["avgTa", "avgPa"] ]

In [ ]:
sub_data.corr()

In [ ]:
sub_data.corr(method='pearson')

In [ ]:
sub_data.corr(method='kendall')

In [ ]:
data = data.drop(['tm', 'stnId'], axis = 1)
corr = data.corr(method='pearson')
corr

In [ ]:
sub_data = data [ ["avgTa", "avgRhm"]]

In [ ]:
sub_data.describe()

In [ ]:
m = len(sub_data)

평균구하기

In [ ]:
s = np.sum(sub_data, axis=0)
mu = s/m
mu

### z-value 
* 평균값과 얼마나 거리가 먼지 계산해주는 통계적인 예측값

>  z = (x – μ) / σ

* x : 개별적으로 갖는 수치
* σ : 표준편차
* μ: 평균

> 임계값보다 큰 값을 이상치(anomaly) 로 정함

* 정상에서 벗어난 데이터를 이상치라고 하며 대부분 데이터가 정상일 수 있지만 그렇지 않을 수도 있음 
* 자료에서 예상과는 다른 패턴을 보이는 개체 또는 자료를 찾기 위해 z-value을 사용할 수 있음

--
* avgTa 필드를 대상으로 z-value 를 구한다


In [ ]:
# 평균
def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

# 편차 
def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

# 표준 편차 
def stddev(data, ddof=0):
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

In [ ]:
def find_anomaly(data,th):
    mean_data =  mean(data)
    std_data =  stddev(data, ddof=1)
    print(f'평균 : {mean_data}')
    print(f'편차 : {std_data}')
    threshold = th
    z_arr = []
    anomalies = []
    for d in data:
        # z값을 계산한 후 임계값보다 큰값을 이상값으로 취함
        z = ((d - mean_data)/std_data)
        z_arr.append(z)
        #print (f'z = {z}, data ={d}')

        if abs(z) > threshold:
            anomalies.append(d)

    print (f'anomalies in this data set : {anomalies}')
    return z_arr



In [ ]:
df = data ["avgTa"]
threshold = 1.5
z_value = find_anomaly(df, threshold)

In [ ]:
z_value

## 상자 그림

* 자료로부터 얻어낸 통계량인 5가지 요약 수치를 가지고 결과를 표현함
* 최솟값, 제 1사분위 $Q_{1}$, 제 2사분위 $Q_{2}$, 제 3사분위 $Q_{3}$, 최댓값


In [ ]:
fig = plt.figure(figsize =(6, 4))
 
# Creating plot
plt.boxplot(z_value)
 
# show plot
plt.show()